In [26]:
# import packages we'll use here
import pandas as pd
import numpy as np
import scipy.optimize as opt
import time
from geopy.distance import vincenty as vc

In [27]:
# Read in the data
ps4_data = pd.read_excel('radio_merger_data.xlsx')

In [28]:
ps4_data.head(n=5)

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,157763.91,80,3,21676,0
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,1472463.20,376,1,11539,0
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,3786333.90,129,1,182265,0
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,473291.74,188,20,203065,0
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,1840579.00,284,0,1493350,0


In [29]:
ps4_data.describe()

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,9.900000e+01,99.000000,99.000000,9.900000e+01,99.000000
mean,2007.545455,25.454545,25.454545,36.909475,-94.392918,37.476019,-94.365015,4.141370e+06,96.353535,18.828283,3.229588e+05,0.020202
std,0.500464,14.709620,14.709620,5.068201,13.957259,4.853933,13.695796,1.471255e+07,82.242463,69.837719,1.098850e+06,0.141407
min,2007.000000,1.000000,1.000000,25.558428,-122.710600,25.369355,-122.433420,5.724072e+04,10.000000,0.000000,2.553000e+03,0.000000
25%,2007.000000,13.000000,13.000000,32.886576,-104.497680,33.687825,-104.876615,4.913584e+05,54.500000,1.000000,2.169800e+04,0.000000
50%,2008.000000,25.000000,25.000000,36.653256,-92.662675,36.900901,-92.122592,1.013110e+06,72.000000,2.000000,4.994800e+04,0.000000
75%,2008.000000,37.500000,37.500000,40.858790,-83.813483,41.145355,-84.212299,2.330153e+06,91.000000,5.500000,1.538550e+05,0.000000
max,2008.000000,54.000000,54.000000,48.704839,-71.411500,48.906401,-70.018443,1.393000e+08,388.000000,591.000000,9.818605e+06,1.000000


In [30]:
# Scale variables
ps4_data['pop_ths_log'] = np.log(ps4_data['population_target'] / 1000)
ps4_data['price_ths_log'] = np.log(ps4_data['price'] / 1000)

In [31]:
# Define a function and then calculate the distance
def distance_calc (row):
    start = (row['buyer_lat'], row['buyer_long'])
    stop = (row['target_lat'], row['target_long'])

    return vc(start, stop).miles

'''
The distance here is measured in miles, it can also be measured in meters.
The value is a little big, so it's better to use log value.

'''

"\nThe distance here is measured in miles, it can also be measured in meters.\nThe value is a little big, so it's better to use log value.\n\n"

In [32]:
ps4_data['distance'] = ps4_data.apply (lambda row: distance_calc (row),axis = 1)
ps4_data['log_distance'] = np.log(ps4_data['distance'])

In [33]:
ps4_data_2007 = ps4_data[(ps4_data['year'] == 2007)].copy()

In [34]:
ps4_data_2007.head(n=5)

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer,pop_ths_log,price_ths_log,distance,log_distance
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,157763.91,80,3,21676,0,3.076206,5.061100,155.126597,5.044242
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,1472463.20,376,1,11539,0,2.445733,7.294692,51.579070,3.943116
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,3786333.90,129,1,182265,0,5.205462,8.239154,44.117249,3.786851
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,473291.74,188,20,203065,0,5.313526,6.159712,193.919359,5.267442
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,1840579.00,284,0,1493350,0,7.308777,7.517835,10.776938,2.377408


In [35]:
ps4_data_2007.describe()
# Data is not unique, two buyers have double targets !!!

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer,pop_ths_log,price_ths_log,distance,log_distance
count,45.0,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,4.500000e+01,45.000000,45.000000,4.500000e+01,45.0,45.000000,45.000000,45.000000,45.000000
mean,2007.0,23.000000,23.000000,37.638507,-97.528344,38.128458,-95.929336,3.233108e+06,89.444444,11.822222,2.651499e+05,0.0,4.185451,7.071652,326.524000,4.510326
std,0.0,13.133926,13.133926,5.007626,14.009829,5.056040,13.740064,5.802951e+06,70.190829,33.748595,5.214337e+05,0.0,1.641610,1.378888,534.441712,1.748008
min,2007.0,1.000000,1.000000,27.689785,-122.710600,28.709332,-122.433420,1.314699e+05,10.000000,0.000000,3.821000e+03,0.0,1.340512,4.878778,1.336767,0.290254
25%,2007.0,12.000000,12.000000,33.786594,-106.882770,33.846663,-104.536810,4.207038e+05,57.000000,1.000000,1.881500e+04,0.0,2.934654,6.041929,23.730377,3.166756
50%,2007.0,23.000000,23.000000,38.440061,-95.020124,38.064413,-93.836006,1.104347e+06,72.000000,2.000000,5.145400e+04,0.0,3.940688,7.007010,78.225356,4.359594
75%,2007.0,34.000000,34.000000,41.054082,-87.599031,41.729101,-86.932437,2.692504e+06,82.000000,4.000000,1.822650e+05,0.0,5.205462,7.898227,270.569690,5.600530
max,2007.0,45.000000,45.000000,48.704839,-71.411500,48.906401,-70.018443,2.524223e+07,376.000000,208.000000,2.189641e+06,0.0,7.691493,10.136274,2231.268415,7.710325


In [38]:
ps4_data_2007.dtypes

year                    int64
buyer_id                int64
target_id               int64
buyer_lat             float64
buyer_long            float64
target_lat            float64
target_long           float64
price                 float64
hhi_target              int64
num_stations_buyer      int64
population_target       int64
corp_owner_buyer        int64
pop_ths_log           float64
price_ths_log         float64
distance              float64
log_distance          float64
match_id               object
dtype: object

In [63]:
ps4_data_2007.head(n=5)

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,...,corp_owner_buyer,pop_ths_log,price_ths_log,distance,log_distance,match_id,ob1,ob2,ob3,ob4
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,157763.91,80,3,...,0,3.076206,5.061100,155.126597,5.044242,11,9.228617,0.0,5.044242,10.937838
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,1472463.20,376,1,...,0,2.445733,7.294692,51.579070,3.943116,22,2.445733,0.0,3.943116,10.937838
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,3786333.90,129,1,...,0,5.205462,8.239154,44.117249,3.786851,33,5.205462,0.0,3.786851,10.937838
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,473291.74,188,20,...,0,5.313526,6.159712,193.919359,5.267442,44,106.270522,0.0,5.267442,10.937838
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,1840579.00,284,0,...,0,7.308777,7.517835,10.776938,2.377408,55,0.000000,0.0,2.377408,10.937838


In [ ]:
# Datframe 1

In [36]:
ps4_data_2007['match_id'] = ps4_data_2007['buyer_id'].apply(str) + ps4_data_2007['target_id'].apply(str)
ps4_data_2007['ob1'] = ps4_data_2007['num_stations_buyer'] * ps4_data_2007['pop_ths_log']

In [41]:
ps4_data_2007['ob2'] = ps4_data_2007['corp_owner_buyer'] * ps4_data_2007['pop_ths_log']

In [42]:
ps4_data_2007['ob3'] = ps4_data_2007['log_distance']

In [ ]:
# Dataframe 2

In [46]:
ps4_data_2007_copy = ps4_data_2007.copy()

In [48]:
ps4_data_2007_copy.describe()

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer,pop_ths_log,price_ths_log,distance,log_distance,ob1,ob2,ob3
count,45.0,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,4.500000e+01,45.000000,45.000000,4.500000e+01,45.0,45.000000,45.000000,45.000000,45.000000,45.000000,45.0,45.000000
mean,2007.0,23.000000,23.000000,37.638507,-97.528344,38.128458,-95.929336,3.233108e+06,89.444444,11.822222,2.651499e+05,0.0,4.185451,7.071652,326.524000,4.510326,66.776867,0.0,4.510326
std,0.0,13.133926,13.133926,5.007626,14.009829,5.056040,13.740064,5.802951e+06,70.190829,33.748595,5.214337e+05,0.0,1.641610,1.378888,534.441712,1.748008,222.120180,0.0,1.748008
min,2007.0,1.000000,1.000000,27.689785,-122.710600,28.709332,-122.433420,1.314699e+05,10.000000,0.000000,3.821000e+03,0.0,1.340512,4.878778,1.336767,0.290254,0.000000,0.0,0.290254
25%,2007.0,12.000000,12.000000,33.786594,-106.882770,33.846663,-104.536810,4.207038e+05,57.000000,1.000000,1.881500e+04,0.0,2.934654,6.041929,23.730377,3.166756,3.253973,0.0,3.166756
50%,2007.0,23.000000,23.000000,38.440061,-95.020124,38.064413,-93.836006,1.104347e+06,72.000000,2.000000,5.145400e+04,0.0,3.940688,7.007010,78.225356,4.359594,7.408281,0.0,4.359594
75%,2007.0,34.000000,34.000000,41.054082,-87.599031,41.729101,-86.932437,2.692504e+06,82.000000,4.000000,1.822650e+05,0.0,5.205462,7.898227,270.569690,5.600530,15.382986,0.0,5.600530
max,2007.0,45.000000,45.000000,48.704839,-71.411500,48.906401,-70.018443,2.524223e+07,376.000000,208.000000,2.189641e+06,0.0,7.691493,10.136274,2231.268415,7.710325,1418.773115,0.0,7.710325


In [107]:
for i in ps4_data_2007['buyer_id']:
    for j in ps4_data_2007['buyer_id']:
        if i != j:
            ob4 = ps4_data_2007['num_stations_buyer'][i - 1] * ps4_data_2007['pop_ths_log'][j - 1]
            print(i, j, ob4)

1 2 7.33719780664
1 3 15.6163850161
1 4 15.9405783745
1 5 21.9263315922
1 6 8.52564464621
1 7 5.556210949
1 8 20.4630737696
1 9 18.893404589
1 10 6.86606696341
1 11 14.082261738
1 12 8.80651334401
1 13 10.1422696523
1 14 8.92551707877
1 15 16.7912298299
1 16 4.02153650539
1 17 8.64920210369
1 18 23.0744786473
1 19 12.4772585632
1 20 15.0909608022
1 21 10.776362177
1 22 7.27653131949
1 23 9.2347004852
1 24 8.24738448858
1 25 10.5953608541
1 26 12.5527843092
1 27 11.9769908804
1 28 23.0744786473
1 29 15.1053982971
1 30 18.5454984265
1 31 11.1494838634
1 32 14.5320651554
1 33 8.56803099114
1 34 16.9222511216
1 35 12.9424075028
1 36 9.76191798572
1 37 11.9444792891
1 38 12.5011117428
1 39 8.80396327214
1 40 20.7951946125
1 41 8.50334362748
1 42 6.22705347841
1 43 19.8243905147
1 44 10.9378375702
1 45 11.8220646139
2 1 3.0762056577
2 3 5.20546167203
2 4 5.31352612483
2 5 7.30877719739
2 6 2.84188154874
2 7 1.85207031633
2 8 6.82102458987
2 9 6.29780152966
2 10 2.2886889878
2 11 4.6940872460

19 16 10.7240973477
19 17 23.0645389432
19 18 61.5319430595
19 20 40.2425621393
19 21 28.7369658054
19 22 19.4040835187
19 23 24.6258679605
19 24 21.9930253029
19 25 28.2542956109
19 26 33.4740914911
19 27 31.9386423479
19 28 61.5319430595
19 29 40.2810621256
19 30 49.4546624706
19 31 29.7319569692
19 32 38.7521737476
19 33 22.848082643
19 34 45.1260029908
19 35 34.513086674
19 36 26.0317812952
19 37 31.8519447709
19 38 33.3362979809
19 39 23.4772353924
19 40 55.4538522999
19 41 22.6755830066
19 42 16.6054759424
19 43 52.8650413725
19 44 29.1675668538
19 45 31.5255056371
20 1 18.4572339462
20 2 14.6743956133
20 3 31.2327700322
20 4 31.881156749
20 5 43.8526631844
20 6 17.0512892924
20 7 11.112421898
20 8 40.9261475392
20 9 37.7868091779
20 10 13.7321339268
20 11 28.1645234761
20 12 17.613026688
20 13 20.2845393045
20 14 17.8510341575
20 15 33.5824596599
20 16 8.04307301079
20 17 17.2984042074
20 18 46.1489572946
20 19 24.9545171264
20 21 21.552724354
20 22 14.553062639
20 23 18.4694009

32 39 2.93465442405
32 40 6.93173153749
32 41 2.83444787583
32 42 2.0756844928
32 43 6.60813017156
32 44 3.64594585673
32 45 3.94068820464
33 1 9.22861697311
33 2 7.33719780664
33 3 15.6163850161
33 4 15.9405783745
33 5 21.9263315922
33 6 8.52564464621
33 7 5.556210949
33 8 20.4630737696
33 9 18.893404589
33 10 6.86606696341
33 11 14.082261738
33 12 8.80651334401
33 13 10.1422696523
33 14 8.92551707877
33 15 16.7912298299
33 16 4.02153650539
33 17 8.64920210369
33 18 23.0744786473
33 19 12.4772585632
33 20 15.0909608022
33 21 10.776362177
33 22 7.27653131949
33 23 9.2347004852
33 24 8.24738448858
33 25 10.5953608541
33 26 12.5527843092
33 27 11.9769908804
33 28 23.0744786473
33 29 15.1053982971
33 30 18.5454984265
33 31 11.1494838634
33 32 14.5320651554
33 34 16.9222511216
33 35 12.9424075028
33 36 9.76191798572
33 37 11.9444792891
33 38 12.5011117428
33 39 8.80396327214
33 40 20.7951946125
33 41 8.50334362748
33 42 6.22705347841
33 43 19.8243905147
33 44 10.9378375702
33 45 11.8220646

ValueError: If using all scalar values, you must pass an index

In [137]:
s = np.empty((2, 3))
s

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [135]:
ps4_2007 = {'ob1': [], 'ob2': [], 'ob3': [], 'ob4': [], 'ob5': [], 'ob6': [],
            'cf1': [], 'cf2': [], 'cf3': [], 'cf4': [], 'cf5': [], 'cf6': []} # define a dataframe to store the final outputs
num_coef = 3 # the number of coeffients
m_y = ps4_data_2007['buyer_id'].count() # the number of observed matches for year y
varp = np.empty((m_y * (m_y - 1) / 2, (num_coef + num_coef) * 2)) # define an array to store outputs

for b in ps4_data_2007['buyer_id']:
    for t in ps4_data_2007['target_id']:
        if b == t:
            ob1 = ps4_data_2007['num_stations_buyer'][i - 1] * ps4_data_2007['pop_ths_log'][j - 1]
            print(i, j, ob1)
        elif j > i:
            ob3 = ps4_data_2007['num_stations_buyer'][i - 1] * ps4_data_2007['pop_ths_log'][j - 1]
            print(i, j, ob3)
        else:
            ob4 = ps4_data_2007['num_stations_buyer'][i - 1] * ps4_data_2007['pop_ths_log'][j - 1]
            print(i, j, ob4)

1 1 9.22861697311
1 2 7.33719780664
1 3 15.6163850161
1 4 15.9405783745
1 5 21.9263315922
1 6 8.52564464621
1 7 5.556210949
1 8 20.4630737696
1 9 18.893404589
1 10 6.86606696341
1 11 14.082261738
1 12 8.80651334401
1 13 10.1422696523
1 14 8.92551707877
1 15 16.7912298299
1 16 4.02153650539
1 17 8.64920210369
1 18 23.0744786473
1 19 12.4772585632
1 20 15.0909608022
1 21 10.776362177
1 22 7.27653131949
1 23 9.2347004852
1 24 8.24738448858
1 25 10.5953608541
1 26 12.5527843092
1 27 11.9769908804
1 28 23.0744786473
1 29 15.1053982971
1 30 18.5454984265
1 31 11.1494838634
1 32 14.5320651554
1 33 8.56803099114
1 34 16.9222511216
1 35 12.9424075028
1 36 9.76191798572
1 37 11.9444792891
1 38 12.5011117428
1 39 8.80396327214
1 40 20.7951946125
1 41 8.50334362748
1 42 6.22705347841
1 43 19.8243905147
1 44 10.9378375702
1 45 11.8220646139
2 1 3.0762056577
2 2 2.44573260221
2 3 5.20546167203
2 4 5.31352612483
2 5 7.30877719739
2 6 2.84188154874
2 7 1.85207031633
2 8 6.82102458987
2 9 6.29780152966

19 6 22.7350523899
19 7 14.8165625307
19 8 54.5681967189
19 9 50.3824122373
19 10 18.3095119024
19 11 37.5526979681
19 12 23.484035584
19 13 27.0460524061
19 14 23.8013788767
19 15 44.7766128798
19 16 10.7240973477
19 17 23.0645389432
19 18 61.5319430595
19 19 33.2726895019
19 20 40.2425621393
19 21 28.7369658054
19 22 19.4040835187
19 23 24.6258679605
19 24 21.9930253029
19 25 28.2542956109
19 26 33.4740914911
19 27 31.9386423479
19 28 61.5319430595
19 29 40.2810621256
19 30 49.4546624706
19 31 29.7319569692
19 32 38.7521737476
19 33 22.848082643
19 34 45.1260029908
19 35 34.513086674
19 36 26.0317812952
19 37 31.8519447709
19 38 33.3362979809
19 39 23.4772353924
19 40 55.4538522999
19 41 22.6755830066
19 42 16.6054759424
19 43 52.8650413725
19 44 29.1675668538
19 45 31.5255056371
20 1 18.4572339462
20 2 14.6743956133
20 3 31.2327700322
20 4 31.881156749
20 5 43.8526631844
20 6 17.0512892924
20 7 11.112421898
20 8 40.9261475392
20 9 37.7868091779
20 10 13.7321339268
20 11 28.164523476

29 45 3.94068820464
30 1 6.15241131541
30 2 4.89146520443
30 3 10.4109233441
30 4 10.6270522497
30 5 14.6175543948
30 6 5.68376309747
30 7 3.70414063266
30 8 13.6420491797
30 9 12.5956030593
30 10 4.5773779756
30 11 9.38817449202
30 12 5.87100889601
30 13 6.76151310151
30 14 5.95034471918
30 15 11.19415322
30 16 2.68102433693
30 17 5.76613473579
30 18 15.3829857649
30 19 8.31817237547
30 20 10.0606405348
30 21 7.18424145134
30 22 4.85102087966
30 23 6.15646699013
30 24 5.49825632572
30 25 7.06357390273
30 26 8.36852287277
30 27 7.98466058696
30 28 15.3829857649
30 29 10.0702655314
30 30 12.3636656177
30 31 7.43298924229
30 32 9.68804343691
30 33 5.71202066076
30 34 11.2815007477
30 35 8.62827166851
30 36 6.50794532381
30 37 7.96298619272
30 38 8.33407449523
30 39 5.86930884809
30 40 13.863463075
30 41 5.66889575165
30 42 4.1513689856
30 43 13.2162603431
30 44 7.29189171346
30 45 7.88137640928
31 1 3.0762056577
31 2 2.44573260221
31 3 5.20546167203
31 4 5.31352612483
31 5 7.30877719739


41 1 33.8382622347
41 2 26.9030586243
41 3 57.2600783923
41 4 58.4487873732
41 5 80.3965491713
41 6 31.2606970361
41 7 20.3727734797
41 8 75.0312704886
41 9 69.2758168262
41 10 25.1755788658
41 11 51.6349597061
41 12 32.290548928
41 13 37.1883220583
41 14 32.7268959555
41 15 61.5678427098
41 16 14.7456338531
41 17 31.7137410469
41 18 84.6064217068
41 19 45.7499480651
41 20 55.3335229415
41 21 39.5133279824
41 22 26.6806148381
41 23 33.8605684457
41 24 30.2404097915
41 25 38.849656465
41 26 46.0268758002
41 27 43.9156332283
41 28 84.6064217068
41 29 55.3864604227
41 30 68.0001608971
41 31 40.8814408326
41 32 53.284238903
41 33 31.4161136342
41 34 62.0482541124
41 35 47.4554941768
41 36 35.793699281
41 37 43.79642406
41 38 45.8374097238
41 39 32.2811986645
41 40 76.2490469124
41 41 31.1789266341
41 42 22.8325294208
41 43 72.6894318872
41 44 40.105404424
41 45 43.3475702511
42 1 3.0762056577
42 2 2.44573260221
42 3 5.20546167203
42 4 5.31352612483
42 5 7.30877719739
42 6 2.84188154874
42 

In [106]:
ps4_data_2007_cf 

NameError: name 'ps4_data_2007_cf' is not defined

In [113]:
z = np.array([0, 0, 0])
for i in ps4_data_2007['buyer_id']:
    for j in ps4_data_2007['buyer_id']:
        if i != j:
            ob4 = ps4_data_2007['num_stations_buyer'][i - 1] * ps4_data_2007['pop_ths_log'][j - 1]
            z = np.vstack([z, [i, j, ob4]])

In [117]:
z1 = np.delete(z, 0, 0)
z1

array([[  1.        ,   2.        ,   7.33719781],
       [  1.        ,   3.        ,  15.61638502],
       [  1.        ,   4.        ,  15.94057837],
       ..., 
       [ 45.        ,  42.        ,   6.22705348],
       [ 45.        ,  43.        ,  19.82439051],
       [ 45.        ,  44.        ,  10.93783757]])

In [120]:
df = pd.DataFrame(z1)
df

,0,1,2
0,1.0,2.0,7.337198
1,1.0,3.0,15.616385
2,1.0,4.0,15.940578
3,1.0,5.0,21.926332
4,1.0,6.0,8.525645
5,1.0,7.0,5.556211
6,1.0,8.0,20.463074
7,1.0,9.0,18.893405
8,1.0,10.0,6.866067
9,1.0,11.0,14.082262


In [ ]:
ps4_data_2007_merge1 = ps4_data_2007.rename(columns = {'num_stations_buyer':'num_stations_buyer_merge1', 'pop_ths_log': 'pop_ths_log_merge1',
                               'corp_owner_buyer': 'corp_owner_buyer_merge1', 'log_distance': 'log_distance_merge1'}, inplace = True)

In [ ]:
ps4_data_2007_merge1

In [ ]:
# Construct another dataframe for MSE